In [1]:
import rpy2
import logging
import warnings
import anndata2ri
import pandas as pd
import scanpy as sc
from tabnanny import verbose
from gsva_prep import prep_gsva
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from matplotlib.pyplot import rcParams
from functions import pathway_analyses

In [2]:
def get_sys_dpi(width, height, diag):
    '''
    obtain dpi of system
    
    w: width in pixels (if unsure, go vist `whatismyscreenresolution.net`)
    h: height in pixels
    d: diagonal in inches
    '''
    w_inches = (diag**2/ (1 + height**2/width**2))**0.5
    return round(width/w_inches)

In [3]:
# # Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

warnings.filterwarnings("ignore", category=PendingDeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

rcParams['figure.dpi'] = get_sys_dpi(1512, 982, 14.125)
#rcParams['figure.figsize']=(4,4) #rescale figures

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()



-----
anndata     0.8.0
scanpy      1.9.3
-----
PIL                         9.5.0
absl                        NA
anndata2ri                  1.1
appnope                     0.1.3
asttokens                   NA
attr                        23.1.0
backcall                    0.2.0
botocore                    1.31.17
cffi                        1.15.1
comm                        0.1.3
cycler                      0.10.0
cython_runtime              NA
dateutil                    2.8.2
debugpy                     1.6.7
decorator                   5.1.1
decoupler                   1.4.0
deprecate                   0.3.2
dot_parser                  NA
executing                   1.2.0
fsspec                      2023.6.0
functions                   NA
google                      NA
gsva_prep                   NA
h5py                        3.9.0
igraph                      0.10.4
ipykernel                   6.23.2
ipywidgets                  8.0.6
jedi                        0.18.2
jinja2      

In [4]:
%%R
suppressPackageStartupMessages({
    library(WGCNA)
    library(Matrix)
    library(viridis)
    library(harmony)
    library(ggpubr)
    library(tictoc)
    library(RColorBrewer)
    library(Hmisc)
    library(corrplot)
    library(grid)
    library(gridExtra)
    library(igraph)
    library(ggrepel)
    library(readxl)
    library(conflicted)



    # single-cell analysis package
    library(Seurat)

    # plotting and data science packages
    library(tidyverse)
    library(cowplot)
    library(patchwork)

    # co-expression network analysis packages:
    library(WGCNA)
    library(hdWGCNA)

    # gene enrichment packages
    library(enrichR)
    library(GeneOverlap)


    library(GSEABase)
    library(GSVA) 
# needs to be run every time you start R and want to use %>%
})

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

# set random seed for reproducibility
set.seed(12345)

# optionally enable multithreading
# enableWGCNAThreads(nThreads = 4)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In addition: Warning messages:
1: package ‘IRanges’ was built under R version 4.3.1 
2: package ‘GSVA’ was built under R version 4.3.1 


## **Prepare data**

Now, we load the preprocessed and annotated data for downstream analysis.

In [5]:
save_prefix = 'leng_sfg'
map_meta = True
deg_method =  'DESeq2-Wald'
test_names = ['late_vs_early', 'early_vs_no', 'late_vs_no', 'ad_vs_no']
filter_genes = "TRUE"
subject_ids_for_study = {'leng_sfg': 'PatientID',
                        'leng_etc': 'PatientID',
                        'seaad_mtg': 'individualID'}

subject_id = subject_ids_for_study[save_prefix]     # for leng this is `PatientID` for mathys is 'Subject', and allen is 'individualID'
gene_celltype_threshold = 0.10          # determines number of cells the gene must be expressed in 
covariates = ['None']                   # list of covariates to be accounted for in regression.
gene_selection = 'custom'               # specifies the gene selection method when setting up seurat object for WGCNA. The th
celltypes = ["Excitatory", "Inhibitory", "Astrocyte", "Microglia", "Oligodendrocyte", "OPC"]
metadata = f'../data/raw/{save_prefix}/{save_prefix}_metadata.csv' 
meta = pd.read_csv(metadata, encoding_errors='ignore')

gene_selection = 'custom'   # specifies the gene selection method when setting up seurat object for WGCNA. The th
                            # Posible values are "custom", "fraction", "variable"
                            # If custom, a list of genes must be passed.

gene_set_select = 'diff_exp'      # If gene_selection = 'custom'. This specifies how to obtain the list of
                                  # genes to pass into `SetupForWGCNA`. # The posible values are "diff_exp", "overlap", "all"

In [6]:
%%R -i subject_id -i gene_celltype_threshold -i celltypes -i test_names -i save_prefix -i gene_selection -i gene_set_select

# Function to handle failure in data loading
handle_loading_failure <- function(cell_type, seurat_obj) {
    cat(paste0('Could not load data for ', toupper(cell_type), '\n'))
    cat(paste0(toupper(cell_type), ' dropped from experiment\n'))
  
    # Remove the failed cell_type from the list
    seurat_obj <- seurat_obj[names(seurat_obj) != cell_type]
  
    # Update the modified seurat_obj in the global environment
    assign("seurat_obj", seurat_obj, envir = .GlobalEnv)
}

seurat_obj <- list()

for (cell_type in celltypes) {
  
    cat(paste0('Loading data for hdWGCNA Experiment in ', toupper(cell_type), '\n'))
  
    # Use tryCatch to load data and handle errors or messages
    tryCatch({
        # Attempt to read the RDS file into the list
        seurat_obj[[cell_type]] <- readRDS(paste0("../results/hdWGCNA/SeuratObject/", save_prefix, '/', cell_type, '_hdWGCNA_object.rds'))
        print(seurat_obj[[cell_type]])
        print('............')
  
    }, error = function(e){
        # Error handling: Remove the cell_type from the list and print an error message
        handle_loading_failure(cell_type, seurat_obj)

    }, message = function(m){
        handle_loading_failure(cell_type, seurat_obj)
    })
}

cat("Loaded available data\n")

Loading data for hdWGCNA Experiment in EXCITATORY


An object of class Seurat 
33694 features across 20301 samples within 1 assay 
Active assay: originalexp (33694 features, 2000 variable features)
 3 dimensional reductions calculated: pca, harmony, umap
[1] "............"
Loading data for hdWGCNA Experiment in INHIBITORY
An object of class Seurat 
33694 features across 7964 samples within 1 assay 
Active assay: originalexp (33694 features, 2000 variable features)
 3 dimensional reductions calculated: pca, harmony, umap
[1] "............"
Loading data for hdWGCNA Experiment in ASTROCYTE
An object of class Seurat 
33694 features across 8025 samples within 1 assay 
Active assay: originalexp (33694 features, 2000 variable features)
 3 dimensional reductions calculated: pca, harmony, umap
[1] "............"
Loading data for hdWGCNA Experiment in MICROGLIA
Could not load data for MICROGLIA
MICROGLIA dropped from experiment
Loading data for hdWGCNA Experiment in OLIGODENDROCYTE
Could not load data for OLIGODENDROCYTE
OLIGODENDROCYTE dropped f

In addition: Warning messages:
1: In gzfile(file, "rb") :
  cannot open compressed file '../results/hdWGCNA/SeuratObject/leng_sfg/Microglia_hdWGCNA_object.rds', probable reason 'No such file or directory'
2: In gzfile(file, "rb") :
  cannot open compressed file '../results/hdWGCNA/SeuratObject/leng_sfg/Oligodendrocyte_hdWGCNA_object.rds', probable reason 'No such file or directory'
3: In gzfile(file, "rb") :
  cannot open compressed file '../results/hdWGCNA/SeuratObject/leng_sfg/OPC_hdWGCNA_object.rds', probable reason 'No such file or directory'


# **hdWGCNA Basic Visualization**

Here we visualize some results using hdWGCNA, and we employ some of Seurat’s built-in plotting tools to visualize our hdWGCNA results.



### **Module Feature Plots**

hdWGCNA includes the ModuleFeaturePlot function to consruct FeaturePlots for each co-expression module colored by each module’s uniquely assigned color.

In [8]:
%%R


fig_dir = paste0("../results/hdWGCNA/FeaturePlots/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in names(seurat_obj)){
    
  print(paste0('Module Feature Plots in hdWGCNA Experiment for ', toupper(cell_type)))
     
  # make a featureplot of hMEs for each module
  plot_list <- ModuleFeaturePlot(
    seurat_obj[[cell_type]],
    features='hMEs', # plot the hMEs
    order=TRUE, # order so the points with highest hMEs are on top
    # raster=TRUE, 
    # raster_dpi=400, 
    # alpha=1,
    # restrict_range=FALSE, 
    # raster_scale=0.25,
  )

  plot_list <- lapply(1:length(plot_list), function(x){
    plot_list[[x]] + NoLegend() + theme(plot.title=element_text(face='plain', vjust=0.25), plot.margin=margin(c(0,0,0,0)))
  })

  pdf(paste0(fig_dir, cell_type, "_featureplot_MEs.pdf"), height=10, width=5)
  print(wrap_plots(plot_list, ncol=3))
  dev.off()
}

[1] "Module Feature Plots in hdWGCNA Experiment for EXCITATORY"
[1] "EXC-M1"
[1] "EXC-M2"
[1] "Module Feature Plots in hdWGCNA Experiment for INHIBITORY"
[1] "INH-M1"
[1] "INH-M2"
[1] "INH-M3"
[1] "INH-M4"
[1] "INH-M5"
[1] "Module Feature Plots in hdWGCNA Experiment for ASTROCYTE"
[1] "AST-M1"
[1] "AST-M2"


We can also plot the hub gene signature score using the same function:

In [10]:
%%R


fig_dir = paste0("../results/hdWGCNA/FeaturePlots/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in names(seurat_obj)){
    
  print(paste0('Module Feature Plots for HUB GENES in hdWGCNA Experiment for ', toupper(cell_type)))
     
  # make a featureplot of hMEs for each module
  plot_list <- ModuleFeaturePlot(
    seurat_obj[[cell_type]],
    features='scores',  # plot the hub gene scores
    order='shuffle', # order so cells are shuffled
    ucell = TRUE, # depending on Seurat vs UCell for gene scoring
    # raster=TRUE, 
    # raster_dpi=400, 
    # alpha=1,
    # restrict_range=FALSE, 
    # raster_scale=0.25,
  )

  plot_list <- lapply(1:length(plot_list), function(x){
    plot_list[[x]] + NoLegend() + theme(plot.title=element_text(face='plain', vjust=0.25), plot.margin=margin(c(0,0,0,0)))
  })

  pdf(paste0(fig_dir, cell_type, "_featureplot_hubs.pdf"), height=10, width=5)
  print(wrap_plots(plot_list, ncol=3))
  dev.off()
}

[1] "Module Feature Plots for HUB GENES in hdWGCNA Experiment for EXCITATORY"
[1] "EXC-M1"
[1] "EXC-M2"
[1] "Module Feature Plots for HUB GENES in hdWGCNA Experiment for INHIBITORY"
[1] "INH-M1"
[1] "INH-M2"
[1] "INH-M3"
[1] "INH-M4"
[1] "INH-M5"
[1] "Module Feature Plots for HUB GENES in hdWGCNA Experiment for ASTROCYTE"
[1] "AST-M1"
[1] "AST-M2"


### **Module Correlations**

hdWGCNA includes the ModuleCorrelogram function to visualize the correlation between each module based on their hMEs, MEs, or hub gene scores using the R package corrplot.

In [12]:
%%R -i seurat_obj -o seurat_obj


fig_dir = paste0("../results/hdWGCNA/ModuleCorrelations/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){

    print(paste0('Module Correlation in hdWGCNA Experiment for ', toupper(cell_type)))
    # plot module correlagram
    pdf(paste0(fig_dir, cell_type, "_correlogram.pdf"), height=10, width=5)
    ModuleCorrelogram(seurat_obj[[cell_type]])
    dev.off()
}

[1] "Module Correlation in hdWGCNA Experiment for EXCITATORY"
[1] "Module Correlation in hdWGCNA Experiment for INHIBITORY"
[1] "Module Correlation in hdWGCNA Experiment for ASTROCYTE"


In addition: Warning messages:
1: In pNew > sig.level :
  longer object length is not a multiple of shorter object length
2: In pNew > sig.level :
  longer object length is not a multiple of shorter object length
3: In pNew > sig.level :
  longer object length is not a multiple of shorter object length


### **Seurat plotting functions**

In [14]:
%%R

mods <- list()

for (cell_type in names(seurat_obj)){

    # get hMEs from seurat object
    MEs <- GetMEs(seurat_obj[[cell_type]], harmonized=TRUE)
    mods[[cell_type]] <- colnames(MEs); 
    mods[[cell_type]] <- mods[[cell_type]][mods[[cell_type]] != 'grey']
    
    
    # add hMEs to Seurat meta-data:
    seurat_obj[[cell_type]]@meta.data <- cbind(seurat_obj[[cell_type]]@meta.data, MEs)
}

Now we can easily use Seurat’s DotPlot function:

In [16]:
%%R -i seurat_obj -i mods -o seurat_obj


fig_dir = paste0("../results/hdWGCNA/DotPlot/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in names(seurat_obj)){

  print(paste0('DotPlot for average expression of MEs in hdWGCNA Experiment for ', toupper(cell_type)))

  # plot with Seurat's DotPlot function
  p <- DotPlot(seurat_obj[[cell_type]], features=mods[[cell_type]], group.by = 'cell_type')

  # flip the x/y axes, rotate the axis labels, and change color scheme:
  p <- p +
    coord_flip() +
    RotatedAxis() +
    scale_color_gradient2(high='red', mid='grey95', low='blue')

  pdf(paste0(fig_dir, cell_type, "_dotplot.pdf"), height=10, width=5)
  print(p)
  dev.off()

}

[1] "DotPlot for average expression of MEs in hdWGCNA Experiment for EXCITATORY"
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
[1] "DotPlot for average expression of MEs in hdWGCNA Experiment for INHIBITORY"
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.
[1] "DotPlot for average expression of MEs in hdWGCNA Experiment for ASTROCYTE"
Scale for colour is already present.
Adding another scale for colour, which will replace the existing scale.


Here is another example where we use Seurat’s VlnPlot function:

In [18]:
# %%R -i seurat_obj -o seurat_obj


# for (cell_type in names(seurat_obj)){

#   print(paste0('ViolinPlot for average expression of MEs in hdWGCNA Experiment for ', toupper(cell_type)))

#   # Plot INH-M4 hME using Seurat VlnPlot function
#   p <- VlnPlot(
#     seurat_obj,
#     features = as.vector(mods[[cell_type]]),
#     group.by = 'cell_type',
#     pt.size = 0 # don't show actual data points
#   )

#   # add box-and-whisker plots on top:
#   p <- p + geom_boxplot(width=.25, fill='white')

#   # change axis labels and remove legend:
#   p <- p + xlab('') + ylab('hME') + NoLegend()

#   # plot output
#   pdf(paste0("../results/hdWGCNA/ViolinPlot/", save_prefix, '_', cell_type, "_violinplot.pdf"), height=10, width=5)
#   print(p)
#   dev.off()

# }


In [19]:
# # convert nested list of Seurat object into Rpy2 object 
# seurat_obj =  robjects.ListVector(
#                         {
#                             cell_type: seurat_obj[cell_type]
                     
#                             for cell_type in seurat_obj.keys()
#                         }
#                     )


## **Overlap With AD Genes**

Next on the co-expression UMAP, we show&quantify the overlap between hub genes that have a known association with AD using the AD databases obtained from the [**Open Targets Platform**](https://platform.opentargets.org/disease/MONDO_0004975/associations), [**KEGG Alzheimer's pathway**](https://www.genome.jp/pathway/hsa05010) and [**Harmonizome (Mayaanlab)**](https://maayanlab.cloud/Harmonizome/gene_set/Alzheimer+Disease/dbGAP+Gene-Trait+Associations) 

In [20]:
KEGG_paths = pathway_analyses.read_pathways('../data/pathway_databases/KEGG_2019_Human.txt')
KEGG_genes = [gene for gene in list(KEGG_paths[(KEGG_paths[0].str.startswith('Alzheimer disease'))].iloc[:, 1:].values[0]) if str(gene)!="nan"]
mayaanlab_genes = pd.read_csv('../data/pathway_databases/AD_genes.csv').Symbol.to_list()
otp_genes = pd.read_csv('../data/pathway_databases/MONDO_0004975-associated-diseases.tsv', sep='\t')
otp_genes = otp_genes[otp_genes.textMining!='No data'].symbol.to_list()

AD_genes = list(set(set(KEGG_genes).union(set(mayaanlab_genes), set(otp_genes))))

In [21]:
%%R -i AD_genes

conflicts_prefer(base::intersect)

# load modules

fig_dir = paste0("../results/hdWGCNA/GeneOverlap/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}


for (cell_type in names(seurat_obj)){

  print(paste0('Estimating AD gene overlap in hdWGCNA Experiment for ', toupper(cell_type)))

  modules <- GetModules(seurat_obj[[cell_type]])
  mods <- levels(modules$module)

  AD_genes <- intersect(as.vector(AD_genes), (GetWGCNAGenes(seurat_obj[[cell_type]])))
 
  # genome.size <- dim(seurat_obj[[cell_type]])[1]
  genome.size <- nrow(modules)
  # genome.size <- length(GetWGCNAGenes(seurat_obj[[cell_type]]))
  # genome.size <- length(c(GetWGCNAGenes(seurat_obj[[cell_type]]), AD_genes))

  overlap_df <- do.call(rbind, lapply(mods, function(cur_mod){

  cur_genes <- modules %>% subset(module == cur_mod) %>% .$gene_name

  cur_overlap <- testGeneOverlap(newGeneOverlap(
      cur_genes,
      as.vector(AD_genes),
      genome.size=genome.size
  ))

  cur_overlap <- data.frame(
    'odds.ratio' = cur_overlap@odds.ratio,
    'pval' = cur_overlap@pval,
    'Jaccard' = cur_overlap@Jaccard,
    'size_intersection' = length(cur_overlap@intersection),
    'module' = cur_mod
  )

    cur_overlap

  })) %>% as.data.frame()

  overlap_df <- overlap_df %>% mutate(fdr=p.adjust(pval, method='fdr'))
  overlap_df <- overlap_df %>% subset(module != 'grey')

  ################################################################################
  # Plot as a lollipop
  ################################################################################

  overlap_df$shape <- ifelse(overlap_df$fdr < 0.05, 21, 4)
  overlap_df <- overlap_df %>% arrange(odds.ratio, descending=TRUE)
  overlap_df$module <- factor(as.character(overlap_df$module), levels=as.character(overlap_df$module))

  mod_colors <- dplyr::select(modules, c(module, color)) %>%
    distinct
  cp <- mod_colors$color; names(cp) <- mod_colors$module

  p <- overlap_df %>%
    ggplot(aes(y=module, x=odds.ratio, size= size_intersection, color=module)) +
    geom_segment(aes(y=module, yend=module, x=0, xend=odds.ratio), size=0.5, color='grey') +
    geom_point() +
    geom_point(shape=overlap_df$shape, color='black', fill=NA) +
    scale_color_manual(values=cp, guide='none') +
    ylab('') + xlab("Odds ratio") +
    scale_x_continuous(breaks = seq(0, 19, 2)) +
    labs(size='Size\nintersection') +
    ggtitle('Overlap with AD genes') +

    theme(
      panel.border = element_rect(size=1, color='black', fill=NA),
      axis.line.y = element_blank(),
      axis.line.x = element_blank(),
      plot.title = element_text(hjust=0.5, face='plain')
    )

  pdf(paste0(fig_dir, cell_type, '_AD_overlap.pdf'), width=4, height=3.5)
  print(p)
  dev.off()
}


[conflicted] Will prefer base::intersect over any other package.
[1] "Estimating AD gene overlap in hdWGCNA Experiment for EXCITATORY"
[1] "Estimating AD gene overlap in hdWGCNA Experiment for INHIBITORY"
[1] "Estimating AD gene overlap in hdWGCNA Experiment for ASTROCYTE"


In addition: Warning messages:
1: Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 
2: The `size` argument of `element_rect()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was
generated. 


## **Overlap With DEGs**

We quantify the overlap between hub genes and DEGs obtained from previous analysis and saved at `../results/{test_name}/{save_prefix}_{deg_method}_degs.xlsx`.

In [23]:
%%R -i deg_method


# load modules
conflicts_prefer(base::intersect)

fig_dir = paste0("../results/hdWGCNA/GeneOverlap/", save_prefix, '/')

if (!dir.exists(fig_dir)) {
  dir.create(fig_dir, recursive=TRUE)
}

for (test in c('ad_vs_no')){
  for (cell_type in names(seurat_obj)){

    print(paste0('Estimating AD gene overlap in hdWGCNA Experiment for ', toupper(cell_type)))

    wgcna_genes <- GetWGCNAGenes(seurat_obj[[cell_type]])
    # hub_genes <- GetHubGenes(seurat_obj[[cell_type]], 25)
    degs <- read_excel(paste0('../results/', test, '/', save_prefix, '/DEG/', deg_method, '_degs.xlsx'), sheet = cell_type)
    degs <- intersect(GetWGCNAGenes(seurat_obj[[cell_type]]), 
                      subset(degs, p_val_adj<0.01 & abs_logFC>0.25)$gene)

    modules <- GetModules(seurat_obj[[cell_type]])
    mods <- levels(modules$module)

    # genome.size <- dim(seurat_obj[[cell_type]])[1]
    genome.size <- nrow(modules)
    # genome.size <- length(GetWGCNAGenes(seurat_obj[[cell_type]]))
    # genome.size <- length(c(GetWGCNAGenes(seurat_obj[[cell_type]]), AD_genes))

    overlap_df <- do.call(rbind, lapply(mods, function(cur_mod){

    cur_genes <- modules %>% subset(module == cur_mod) %>% .$gene_name 
    # cur_genes <- intersect(cur_genes, GetHubGenes(seurat_obj[[cell_type]], 25))
    
    cur_overlap <- testGeneOverlap(newGeneOverlap(
        cur_genes,
        as.vector(degs),
        genome.size=genome.size
    ))

    cur_overlap <- data.frame(
      'odds.ratio' = cur_overlap@odds.ratio,
      'pval' = cur_overlap@pval,
      'Jaccard' = cur_overlap@Jaccard,
      'size_intersection' = length(cur_overlap@intersection),
      'module' = cur_mod
    )

      cur_overlap

    })) %>% as.data.frame()

    overlap_df <- overlap_df %>% mutate(fdr=p.adjust(pval, method='fdr'))
    overlap_df <- overlap_df %>% subset(module != 'grey')

    ################################################################################
    # Plot as a lollipop
    ################################################################################

    overlap_df$shape <- ifelse(overlap_df$fdr < 0.05, 21, 4)
    overlap_df <- overlap_df %>% arrange(odds.ratio, descending=TRUE)
    overlap_df$module <- factor(as.character(overlap_df$module), levels=as.character(overlap_df$module))

    mod_colors <- dplyr::select(modules, c(module, color)) %>%
      distinct
    cp <- mod_colors$color; names(cp) <- mod_colors$module

    p <- overlap_df %>%
      ggplot(aes(y=module, x=odds.ratio, size= size_intersection, color=module)) +
      geom_segment(aes(y=module, yend=module, x=0, xend=odds.ratio), size=0.5, color='grey') +
      geom_point() +
      geom_point(shape=overlap_df$shape, color='black', fill=NA) +
      scale_color_manual(values=cp, guide='none') +
      ylab('') + xlab("Odds ratio") +
      scale_x_continuous(breaks = seq(0, 19, 2)) +
      labs(size='Size\nintersection') +
      ggtitle('Overlap with DEGs') +

      theme(
        panel.border = element_rect(size=1, color='black', fill=NA),
        axis.line.y = element_blank(),
        axis.line.x = element_blank(),
        plot.title = element_text(hjust=0.5, face='plain')
      )

    pdf(paste0(fig_dir, test, '_', cell_type, '_DEG_overlap.pdf'), width=4, height=3.5)
    print(p)
    dev.off()
  }
}

[conflicted] Removing existing preference.
[conflicted] Will prefer base::intersect over any other package.
[1] "Estimating AD gene overlap in hdWGCNA Experiment for EXCITATORY"
New names:
• `` -> `...1`
[1] "Estimating AD gene overlap in hdWGCNA Experiment for INHIBITORY"
New names:
• `` -> `...1`
[1] "Estimating AD gene overlap in hdWGCNA Experiment for ASTROCYTE"
New names:
• `` -> `...1`


In addition: There were 26 warnings (use warnings() to see them)


# **Save Seurat Object**

In [25]:
%%R

dat_dir = paste0("../results/hdWGCNA/SeuratObject/", save_prefix, '/')

if (!dir.exists(dat_dir)) {
  dir.create(dat_dir, recursive=TRUE)
}

for (cell_type in names(seurat_obj)){
    
    print(paste0('Saving hdWGCNA object in hdWGCNA Experiment for ', toupper(cell_type)))

    saveRDS(seurat_obj[[cell_type]], file=paste0(dat_dir, cell_type, '_hdWGCNA_object.rds'))

}

[1] "Saving hdWGCNA object in hdWGCNA Experiment for EXCITATORY"
[1] "Saving hdWGCNA object in hdWGCNA Experiment for INHIBITORY"
[1] "Saving hdWGCNA object in hdWGCNA Experiment for ASTROCYTE"
